In [ ]:
!pip install dicttoxml

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import cv2
from dicttoxml import dicttoxml

In [ ]:
# unzip image file
!unzip /content/drive/MyDrive/img.zip

In [ ]:
# list_bbox_inshop data
data = []
with open('/content/drive/MyDrive/In-shop Clothes Retrieval Benchmark/Anno/list_bbox_inshop.txt', 'r') as f:
    for line in f.readlines():
        line = line.strip('\n')
        data.append(line.split())
del data[0], data[0]

In [ ]:
# # description data list_description_inshop.json
description = pd.read_json('/content/drive/MyDrive/In-shop Clothes Retrieval Benchmark/Anno/list_description_inshop.json')

In [133]:
# clothes colors
colors = []
with open('/content/drive/MyDrive/In-shop Clothes Retrieval Benchmark/Anno/attributes/list_color_cloth.txt', 'r') as f:
    for line in f.readlines():
        line = line.strip('\n')
        colors.append(line.split())
del colors[0], colors[0]

for i in range(len(colors)):
    colors[i].remove(colors[i][0])

In [151]:
# list of attributes
list_attr_cloth = []
with open('/content/drive/MyDrive/In-shop Clothes Retrieval Benchmark/Anno/attributes/list_attr_cloth.txt', 'r') as f:
    for line in f.readlines():
        line = line.strip('\n')
        list_attr_cloth.append(line.split())
del list_attr_cloth[0], list_attr_cloth[0]

In [152]:
# list of attributes id
list_attr_items = []
with open('/content/drive/MyDrive/In-shop Clothes Retrieval Benchmark/Anno/attributes/list_attr_items.txt', 'r') as f:
    for line in f.readlines():
        line = line.strip('\n')
        list_attr_items.append(line.split())
del list_attr_items[0], list_attr_items[0]

In [166]:
attribute_dict = {}
for i in range(len(list_attr_items)):
    att_id = []
    att_list = []
    for attr in range(len(list_attr_items[i][1:])):
        if list_attr_items[i][attr] == '1':
            att_id.append(attr-1)
    for id in att_id:
        att_list.append(list_attr_cloth[id][0])
    attribute = ", ".join(att_list)

    attribute_dict[list_attr_items[i][0]] = attribute

In [ ]:
for i in range(len(data)):
    img = cv2.imread(data[i][0]) # data[i][0] is the image path
    cv2.imwrite(f'/content/drive/MyDrive/clothes_data/imgs/{i}.jpg', img)

    dic = {'filename': f'{i}.jpg',  # name of file
           'object': {'gender': data[i][0].split('/')[1], # MEN or WOMAN
                      'name': data[i][0].split('/')[2], # name of cloth such as Blouses_Shirts, Dress, Sweaters, etc
                      'pose': data[i][0].split('/')[4].split('_')[2].split('.')[0], # position(front, back, ...)
                      'bndbox': {'xmin': data[i][3],
                                 'ymin': data[i][4],
                                 'xmax': data[i][5],
                                 'ymax': data[i][6],
                                 },
                      'color': " ".join(colors[i]),
                      'description': [description.loc[item][2] for item in range(len(description)) if data[i][0].split('/')[3] == description.loc[item][0]][0],
                      'attribute_name': attribute_dict[data[i][0].split('/')[3]]
                      }
           }
    xml = dicttoxml(dic, custom_root='annotation', attr_type = False)
    xml_decode = xml.decode()

    xmlfile = open(f"/content/drive/MyDrive/clothes_data/{i}.xml", "w")
    xmlfile.write(xml_decode)
    xmlfile.close()